In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
import time
import datetime
import copy


from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [2]:
# user_spec, loan_result 테이블 불러오기 및 lonapply_insert_time type 변경
user_spec = pd.read_csv('./user_spec.csv')
loan_result = pd.read_csv('./loan_result.csv')
loan_result['loanapply_insert_time'] = pd.to_datetime(loan_result['loanapply_insert_time'])

#### gender, age, amt, credit_score, employment_period 변수 제외한 결측치 처리

In [3]:
def cem_var_preproc(x):
    if len(str(x)) == 10:
        return str(x)[:6] + '01'
    elif len(str(x)) == 8:
        return str(x)[:6] + '01'
    elif str(x) == 'nan':
        return np.nan
    else: 
        print(x)
        return x

# datetime 형변환
user_spec['company_enter_month'] = pd.to_datetime(user_spec['company_enter_month'].apply(cem_var_preproc))
user_spec['insert_time'] = pd.to_datetime(user_spec.insert_time)

# employment_period, age 변수 생성
user_spec['employment_period'] = (user_spec['insert_time'] - user_spec['company_enter_month']).dt.days
user_spec['age'] = user_spec['insert_time'].dt.year - user_spec['birth_year']

# company_enter_month, birth_year drop
user_spec = user_spec.drop(columns = ['company_enter_month', 'birth_year'])

In [4]:
# employment_period가 음수인 사람 같은 user_id의 값으로 대체 - 잘못 입력한 값
row_idx = np.where(user_spec['employment_period'] < 0)[0]
col_idx = np.where(user_spec.columns == 'employment_period')[0][0]
user_spec.iloc[row_idx, col_idx] = 136

In [5]:
# 19세 미만 변수 삭제
print("19세 미만 변수 삭제 전 min / max: ", user_spec.age.min(), user_spec.age.max())
user_spec = pd.concat([user_spec[user_spec.age >= 19], user_spec[user_spec.age.isna()]])
print("19세 미만 변수 삭제 후min / max: ", user_spec.age.min(), user_spec.age.max())

19세 미만 변수 삭제 전 min / max:  14.0 95.0
19세 미만 변수 삭제 후min / max:  19.0 95.0


In [6]:
# income_type, employment_type, houseown_type, desired_amount, purpose nan인 데이터 삭제
user_spec = user_spec[~user_spec.income_type.isna()]

In [7]:
# amt와 cnt가 모두 nan이면 0으로 채운다.
amt_idx = np.where(user_spec.columns == 'existing_loan_amt')[0][0]
cnt_idx = np.where(user_spec.columns == 'existing_loan_cnt')[0][0]
nan_idx = np.where((user_spec.existing_loan_cnt.isna()) & (user_spec.existing_loan_amt.isna()))[0]
user_spec.iloc[nan_idx,amt_idx] = 0
user_spec.iloc[nan_idx,cnt_idx] = 0

In [8]:
# 개인회생 0, 1 조합 삭제
user_spec = user_spec[~((user_spec.personal_rehabilitation_yn == 0)&(user_spec.personal_rehabilitation_complete_yn == 1))]

In [9]:
# 개인회생 nan, nan인 데이터 0으로 처리
yn_idx = np.where(user_spec.columns == 'personal_rehabilitation_yn')[0][0]
cyn_idx = np.where(user_spec.columns == 'personal_rehabilitation_complete_yn')[0][0]
nan_idx = np.where(user_spec.personal_rehabilitation_yn.isna())[0]
user_spec.iloc[nan_idx,yn_idx] = 0
user_spec.iloc[nan_idx,cyn_idx] = 0 

In [10]:
# 개인회생 0, nan인 데이터 0으로 처리
cyn_idx = np.where(user_spec.columns == 'personal_rehabilitation_complete_yn')[0][0]
nan_idx = np.where(user_spec.personal_rehabilitation_complete_yn.isna())[0]
user_spec.iloc[nan_idx,cyn_idx] = 0 

In [11]:
# yearly_income 결측치 데이터 목록
user_spec[user_spec.yearly_income.isna()]

,application_id,user_id,gender,insert_time,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,employment_period,age
165422,1026915,861363,1.0,2022-06-30 21:11:51,NaN,NaN,OTHERINCOME,기타,전월세,5000000.0,생활비,0.0,0.0,0.0,0.0,NaN,27.0
233316,2045613,702899,1.0,2022-06-30 18:37:04,520.0,NaN,FREELANCER,기타,전월세,5000000.0,전월세보증금,0.0,0.0,1.0,1000000.0,29.0,27.0
597591,391560,329226,0.0,2022-06-30 19:27:28,590.0,NaN,OTHERINCOME,기타,전월세,20000000.0,생활비,0.0,0.0,9.0,27000000.0,NaN,26.0
597613,341149,670502,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0,NaN,41.0
971960,1330905,771592,1.0,2022-06-30 18:57:05,750.0,NaN,OTHERINCOME,기타,자가,3000000.0,생활비,0.0,0.0,1.0,3000000.0,NaN,19.0


In [12]:
# 같은 user_id 정보가 없는 데이터 (1개) 삭제
user_spec = user_spec[~(user_spec.user_id == 861363)]

In [13]:
# 같은 user_id 정보가 있는 데이터 mean()으로 대체
col_idx = np.where(user_spec.columns == 'yearly_income')[0][0]
# 1
row_idx = np.where(user_spec.application_id == 2045613)[0]
user_spec.iloc[row_idx, col_idx] = user_spec[user_spec.user_id == 702899].yearly_income.mean()
# 2
row_idx = np.where(user_spec.application_id == 391560)[0]
user_spec.iloc[row_idx, col_idx] = user_spec[user_spec.user_id == 329226].yearly_income.mean()
# 3
row_idx = np.where(user_spec.application_id == 341149)[0]
user_spec.iloc[row_idx, col_idx] = user_spec[user_spec.user_id == 670502].yearly_income.mean()
# 4
row_idx = np.where(user_spec.application_id == 1330905)[0]
user_spec.iloc[row_idx, col_idx] = user_spec[user_spec.user_id == 771592].yearly_income.mean()

In [14]:
user_spec.isna().sum()

application_id                              0
user_id                                     0
gender                                  12959
insert_time                                 0
credit_score                           105014
yearly_income                               0
income_type                                 0
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn                  0
personal_rehabilitation_complete_yn         0
existing_loan_cnt                           0
existing_loan_amt                      115148
employment_period                      171578
age                                     12959
dtype: int64

- LIVING -> 생활비
- ETC -> 기타
- HOUSEDEPOSIT -> 전월세보증금
- BUYHOUSE -> 주택구입
- BUYCAR -> 자동차구입
- INVEST -> 투자
- BUSINESS -> 사업자금
- SWITCHLOAN -> 대환대출

In [15]:
# purpose 영어, 한글 통일
user_spec.purpose.unique()

array(['기타', '대환대출', '생활비', '사업자금', '주택구입', '전월세보증금', '투자', 'LIVING',
       'SWITCHLOAN', 'ETC', 'INVEST', '자동차구입', 'BUSINESS', 'BUYCAR',
       'HOUSEDEPOSIT', 'BUYHOUSE'], dtype=object)

In [16]:
user_spec.purpose = user_spec.purpose.replace(['LIVING', 'ETC', 'HOUSEDEPOSIT', 'BUYHOUSE', 'BUYCAR', 'INVEST', 'BUSINESS', 'SWITCHLOAN'],
                  ['생활비', '기타', '전월세보증금', '주택구입', '자동차구입', '투자', '사업자금', '대환대출'])
user_spec.purpose.unique()

array(['기타', '대환대출', '생활비', '사업자금', '주택구입', '전월세보증금', '투자', '자동차구입'],
      dtype=object)

In [17]:
# bank_id == 16 인 데이터 삭제
loan_result = loan_result[~(loan_result.bank_id == 16)]

In [18]:
# loan_result랑 merge
user_loan = pd.merge(loan_result, user_spec, on = 'application_id')
print(user_loan.shape)
print(user_loan.isna().sum())
user_loan.head()

(13527229, 23)
application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
user_id                                      0
gender                                  128096
insert_time                                  0
credit_score                           1509274
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                      120444

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,gender,insert_time,...,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,employment_period,age
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,430982,1.0,2022-06-07 13:05:39,...,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,126.0,26.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,430982,1.0,2022-06-07 13:05:39,...,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,126.0,26.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,430982,1.0,2022-06-07 13:05:39,...,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,126.0,26.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,430982,1.0,2022-06-07 13:05:39,...,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,126.0,26.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,430982,1.0,2022-06-07 13:05:39,...,정규직,자가,25000000.0,대환대출,0.0,0.0,2.0,15000000.0,126.0,26.0


In [19]:
# month, day 변수 생성 (loanapply_insert_time 기준)
user_loan['month'] = user_loan['loanapply_insert_time'].dt.month
user_loan['day'] = user_loan['loanapply_insert_time'].dt.day

In [20]:
print(user_loan.shape)

(13527229, 25)


In [21]:
# get_dummies
cat_features = ['income_type', 'employment_type', 'houseown_type', 'purpose']
user_loan = pd.get_dummies(user_loan, prefix_sep = "_", columns = cat_features, drop_first = True)

#### 결측을 나타내는 변수 추가

In [22]:
user_loan['gen_age_nan'] = user_loan.gender.isna()
user_loan['credit_nan'] = user_loan.credit_score.isna()
user_loan['amt_nan'] = user_loan.existing_loan_amt.isna()
user_loan['period_nan'] = user_loan.employment_period.isna()

#### 전체 데이터 수치형 변수 impute (MICE - extra tree)

- imp_num_all.pkl : 수치형 결측치 변수들을 MICE로 채운 파일입니다.

In [23]:
drop_columns = ['application_id', 'loanapply_insert_time', 'bank_id','product_id',
                 'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'gender',
                 'insert_time','month','day', 'gen_age_nan', 'credit_nan', 'amt_nan','period_nan']

all_for_impute = user_loan.drop(columns = drop_columns).drop_duplicates(keep = 'first')
all_for_impute_col_name = all_for_impute.columns

In [24]:
start = time.time()
et = ExtraTreesRegressor(n_estimators=10, random_state=1004)
imp = IterativeImputer(estimator = et, max_iter=10, tol=0.1
                    ,skip_complete = True, random_state = 1004, min_value = 0)
imp.fit(all_for_impute)
end = time.time()
print(datetime.timedelta(seconds = end-start))

0:10:23.377082


In [25]:
start = time.time()
imp_all = imp.transform(user_loan.drop(columns = drop_columns))
end = time.time()
print(datetime.timedelta(seconds = end-start))

0:01:45.626487


In [26]:
imp_all = pd.DataFrame(columns = all_for_impute_col_name, index = user_loan.index, data = imp_all)

In [27]:
imp_all = pd.concat([imp_all, user_loan.loc[:,drop_columns]], axis = 1)
print(imp_all.shape)

(13527229, 43)


In [28]:
imp_all.isna().sum()

credit_score                                 0
yearly_income                                0
desired_amount                               0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
employment_period                            0
age                                          0
income_type_EARNEDINCOME2                    0
income_type_FREELANCER                       0
income_type_OTHERINCOME                      0
income_type_PRACTITIONER                     0
income_type_PRIVATEBUSINESS                  0
employment_type_기타                           0
employment_type_일용직                          0
employment_type_정규직                          0
houseown_type_배우자                            0
houseown_type_자가                             0
houseown_type_전월세                            0
purpose_대환대출                                 0
purpose_사업자금 

In [29]:
del et
del imp
gc.collect()

287

In [30]:
imp_all.iloc[:, 3:5] = imp_all.iloc[:, 3:5].astype('uint8')
imp_all.iloc[:, 9:27] = imp_all.iloc[:, 9:27].astype('uint8')

#### gender 결측치 처리(1에 속할 확률으로 대체)

In [31]:
unusing_features = ['application_id', 'loanapply_insert_time', 'product_id', 
                    'user_id', 'insert_time','loan_limit', 'loan_rate', 
                    'is_applied','month', 'day', 'bank_id',
                    'gen_age_nan', 'credit_nan', 'amt_nan','period_nan']
tmp_all = imp_all.drop(columns = unusing_features)
print(tmp_all.shape)

(13527229, 28)


In [32]:
tmp_all_X = tmp_all[~tmp_all.gender.isna()].drop(columns = ['gender'])
tmp_all_y = tmp_all[~tmp_all.gender.isna()]['gender']
print(tmp_all_X.shape)
print(tmp_all_y.shape)

(13399133, 27)
(13399133,)


In [33]:
# 모델 훈련 (logistic regression)
start = time.time()
lr = LogisticRegression(class_weight = 'balanced')
lr.fit(tmp_all_X, tmp_all_y)
end = time.time()
print(datetime.timedelta(seconds = end - start))

0:01:32.376201


In [34]:
all_nan_idx = np.where(imp_all.gender.isna())[0]
col_idx = np.where(imp_all.columns == 'gender')[0][0]

In [35]:
unusing_features = ['application_id', 'loanapply_insert_time', 'product_id', 
                    'user_id', 'insert_time','loan_limit', 'loan_rate', 
                    'is_applied','month', 'day', 'bank_id','gender',
                   'gen_age_nan', 'credit_nan', 'amt_nan','period_nan']
imp_gender_prob = lr.predict_proba(imp_all.iloc[all_nan_idx].drop(columns = unusing_features))

In [36]:
imp_all.iloc[all_nan_idx, col_idx] = imp_gender_prob[:,1]

In [37]:
imp_all.isna().sum()

credit_score                                 0
yearly_income                                0
desired_amount                               0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
employment_period                            0
age                                          0
income_type_EARNEDINCOME2                    0
income_type_FREELANCER                       0
income_type_OTHERINCOME                      0
income_type_PRACTITIONER                     0
income_type_PRIVATEBUSINESS                  0
employment_type_기타                           0
employment_type_일용직                          0
employment_type_정규직                          0
houseown_type_배우자                            0
houseown_type_자가                             0
houseown_type_전월세                            0
purpose_대환대출                                 0
purpose_사업자금 

In [38]:
# age 범주화
def age_to_cat(age):
    if   age < 30: return 2
    elif age < 40: return 3
    elif age < 50: return 4
    elif age < 60: return 5
    elif age < 70: return 6
    elif age < 80: return 7
    else: return 8

In [39]:
imp_all['age_cat'] = imp_all.age.apply(age_to_cat)

In [40]:
# bank_id 전처리

class make_bank_apply_prop():
    
    # fit
    def make_bank_dic(self, X_train, y_train):
        self.bank_dic = {}
        tmp = pd.concat([X_train, y_train], axis = 1)[['bank_id', 'is_applied']].value_counts().reset_index(drop = False)
        bank_id_unq = tmp.bank_id.unique()
        for bid in bank_id_unq:
            tmp_row = tmp[(tmp.bank_id == bid)&(tmp.is_applied == 1)].shape[0]
            if tmp_row == 0:
                count_app = 0
            else:
                count_app = int(tmp[(tmp.bank_id == bid)&(tmp.is_applied == 1)].iloc[:,2])
            count_all = sum(tmp[(tmp.bank_id == bid)].iloc[:,2])
            self.bank_dic[bid] = count_app / count_all
        #return self.bank_dic
    
    # transform using apply function
    def add_bank_prop(self, bank_id):
        # 만약 key가 존재하지 않으면 평균값 반환
        if bank_id in self.bank_dic.keys():
            return self.bank_dic[bank_id]
        else:
            return np.array(list(self.bank_dic.values())).mean()

In [41]:
makebank = make_bank_apply_prop()
makebank.make_bank_dic(imp_all.drop(columns = ['is_applied']), imp_all[['is_applied']]) # imp_all로 fit
imp_all['bank_apply_prop'] = imp_all['bank_id'].apply(makebank.add_bank_prop)

In [42]:
# 모든 결측치 처리한 데이터 저장
imp_all.to_pickle('./final_data/imp_all.pkl')

#### train, valid, test, 6month data split

In [43]:
# 6월 데이터
imp_all_6month = imp_all[imp_all.month == 6]

In [44]:
# 6월 외의 데이터
imp_all = imp_all[imp_all.month != 6]

In [45]:
# train, valid(20000), test(20000) split

In [46]:
# idx 생성
np.random.seed(1004)
num3 = (imp_all.month == 3).sum()
num4 = (imp_all.month == 4).sum()
num5 = (imp_all.month == 5).sum()
idx3 = np.random.choice(num3, size = 12000, replace = False)
idx4 = np.random.choice(num4, size = 12000, replace = False)
idx5 = np.random.choice(num5, size = 16000, replace = False)
not_idx3 = np.array(list(set(range(num3)) - set(idx3)))
not_idx4 = np.array(list(set(range(num4)) - set(idx4)))
not_idx5 = np.array(list(set(range(num5)) - set(idx5)))

In [47]:
# train, valid, test 분리
train3 = imp_all[imp_all.month == 3].iloc[not_idx3]
train4 = imp_all[imp_all.month == 4].iloc[not_idx4]
train5 = imp_all[imp_all.month == 5].iloc[not_idx5]
valid3 = imp_all[imp_all.month == 3].iloc[idx3[:6000]]
valid4 = imp_all[imp_all.month == 4].iloc[idx4[:6000]]
valid5 = imp_all[imp_all.month == 5].iloc[idx5[:8000]]
test3 = imp_all[imp_all.month == 3].iloc[idx3[6000:]]
test4 = imp_all[imp_all.month == 4].iloc[idx4[6000:]]
test5 = imp_all[imp_all.month == 5].iloc[idx5[8000:]]
real_train = pd.concat([train3, train4, train5])
real_valid = pd.concat([valid3, valid4, valid5])
real_test = pd.concat([test3, test4, test5])
print(real_train.shape)
print(real_valid.shape)
print(real_test.shape)

(10229990, 45)
(20000, 45)
(20000, 45)


In [48]:
del idx3
del idx4
del idx5
del not_idx3
del not_idx4
del not_idx5
del train3
del train4
del train5
del valid3
del valid4
del valid5
del test3
del test4
del test5
gc.collect()

16

In [49]:
real_train.to_pickle('./final_data/real_train.pkl')
real_valid.to_pickle('./final_data/real_valid.pkl')
real_test.to_pickle('./final_data/real_test.pkl')
imp_all_6month.to_pickle('./final_data/real_6month.pkl')

In [50]:
del cat_features
del imp_all_6month
del real_train
del real_valid
del real_test
gc.collect()

32

In [51]:
# 확인
real_train = pd.read_pickle('./final_data/real_train.pkl')
real_valid = pd.read_pickle('./final_data/real_valid.pkl')
real_test  = pd.read_pickle('./final_data/real_test.pkl')
real_6month= pd.read_pickle('./final_data/real_6month.pkl')

In [52]:
print(real_train.shape)
print(real_valid.shape)
print(real_test.shape)
print(real_6month.shape)

(10229990, 45)
(20000, 45)
(20000, 45)
(3257239, 45)


In [53]:
print(real_train.isna().sum())
print(real_valid.isna().sum())
print(real_test.isna().sum())
print(real_6month.isna().sum())

credit_score                              0
yearly_income                             0
desired_amount                            0
personal_rehabilitation_yn                0
personal_rehabilitation_complete_yn       0
existing_loan_cnt                         0
existing_loan_amt                         0
employment_period                         0
age                                       0
income_type_EARNEDINCOME2                 0
income_type_FREELANCER                    0
income_type_OTHERINCOME                   0
income_type_PRACTITIONER                  0
income_type_PRIVATEBUSINESS               0
employment_type_기타                        0
employment_type_일용직                       0
employment_type_정규직                       0
houseown_type_배우자                         0
houseown_type_자가                          0
houseown_type_전월세                         0
purpose_대환대출                              0
purpose_사업자금                              0
purpose_생활비                     